In [ ]:
import torch

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as pl
import scipy.signal as sg

pl.style.use('fivethirtyeight')
mpl.rcParams['axes.facecolor']='white'  
mpl.rcParams['figure.facecolor'] = '1'

from scipy.ndimage  import gaussian_filter1d

import json

In [ ]:
from masking import *
from latencies import *
from excitation import *
from deconv import *
from ur import *
from tuning import *
from test import *

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

### Simulate CAPs

In [ ]:
#masking degree function
mdFunc=SigmoidMaskingDegreeFunction(30, 2*1/10., requires_grad=False)

#latencies
lat0=Eggermont1976clickLatencies80dB #latencies

#Unitary response
ur0=URWang1979m
t=np.linspace(5e-4, 10e-3, num=500)
u00=ur0.u(t-2e-3) #shift
sig=0.2e-3 #std of gaussian kernel in s
sig2=sig/(t[1]-t[0])
u0 = gaussian_filter1d(u00, sigma=sig2)
u0/=np.amax(np.abs(u0))
u0*=0.5*np.amax(np.abs(u00))

#tuning
BW10_0=500
BW10Func=constant_BW10(BW10_0, requires_grad=False)

plotMaskingDegreeFunc(mdFunc)
plotLatencies(lat0)

In [ ]:
pl.plot(t*1e3, u00, label='unitary response')
pl.plot(t*1e3, u0, label='ur + nPSHT (blur)',  color='orange')
pl.legend(loc='lower right')
pl.title("Unitary response")
pl.xlabel('t (ms)')

Example: two maskers

In [ ]:
json_txt='''{
    "type":"noise-bands", 
    "comment":"created with noiseBandMaskerDesigner", 
    "n_bands":3, 
    "bands":[
        {
            "amplitude":0.02512, 
            "fc_low":2662.5, 
            "fc_high":4162.5
        }, 
        {
            "amplitude":0.1, 
            "fc_low":4800, 
            "fc_high":5200
        }, 
        {
            "amplitude":0.01585, 
            "fc_low":5375, 
            "fc_high":6808.5
        }
    ], 
    "name":"37-2bands-4769-1213-3412-1500-32dB"
}'''

json_txt2='''{
    "type":"noise-bands", 
    "comment":"created with noiseBandMaskerDesigner", 
    "n_bands":1, 
    "bands":{
        "amplitude":0.1, 
        "fc_low":4800, 
        "fc_high":12000
    }, 
    "name":"4-hp-4800Hz"
}'''
#stim_struct=json.loads(json_txt)

maskingConditions=MaskingConditions.from_json_strings([json_txt, json_txt2])
#maskingConditions.add_json_strings([json_txt])

In [ ]:
#maskingConditions.get_tensor_lists()
maskingConditions

In [ ]:
pl.figure(figsize=(11,5))
plotMaskingExcitations( BW10Func, maskingConditions)

In [ ]:
t=torch.linspace(0.5, 7, 500)*1e-3
f=lat0.f_from_t(t)

sq_excs=get_sq_masking_excitation_patterns_maskCond(f, BW10Func, maskingConditions, filter_model='gaussian')

pl.figure()
for sq_exc in sq_excs:
    pl.plot(t, np.sqrt(sq_exc))

### Interactive plots

In [ ]:
maskingConditions.set_amp0(0.5)

In [ ]:
#NB: withdraw 'manual' for continuous update
@manual_interact( BW0=widgets.IntSlider(value=500, min=100, max=2000, step=50), BW1=widgets.IntSlider(value=500, min=100, max=2000, step=50))
def plotMaskingExcitations_interact(BW0, BW1):
    BW10func0=constant_BW10(BW0)
    BW10func1=constant_BW10(BW1)
    
    pl.figure(figsize=(12, 5))
    axlist=plotMaskingExcitations( BW10func0, maskingConditions)
    plotMaskingExcitations( BW10func1, maskingConditions, axlist=axlist)

In [ ]:

@manual_interact(I0=(30, 100), I1=(30,100), BW0=widgets.IntSlider(value=500, min=100, max=2000, step=50), BW1=widgets.IntSlider(value=500, min=100, max=2000, step=50))
def plotMaskingAmountExcitations_interact(I0, I1, BW0, BW1):
    BW10func0=constant_BW10(BW0)
    BW10func1=constant_BW10(BW1)
    
    pl.figure(figsize=(12, 5))
    
    maskingConditions.set_amp0_dB(I0)
    axlist=plotMaskingAmountExcitations( BW10func0, maskingConditions, mdFunc)
    
    maskingConditions.set_amp0_dB(I1)
    axlist=plotMaskingAmountExcitations( BW10func1, maskingConditions, mdFunc, axlist=axlist)